<a href="https://colab.research.google.com/github/saktheeswaranswan/random-math-addition-data-for-rnn-generator/blob/main/image_transparent_try_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import cv2
import numpy as np
import os

def remove_background(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    foreground = cv2.bitwise_and(img, img, mask=mask)

    background = np.full(img.shape, 255, dtype=np.uint8)
    background = cv2.bitwise_and(background, background, mask=cv2.bitwise_not(mask))

    transparent = cv2.addWeighted(background, 0.5, foreground, 0.5, 0)
    transparent = cv2.cvtColor(transparent, cv2.COLOR_BGR2RGBA)
    _, alpha = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)
    b, g, r, a = cv2.split(transparent)
    rgba = [b, g, r, alpha]
    transparent = cv2.merge(rgba, 4)

    return transparent

def process_images(image_folder):
    for i, filename in enumerate(os.listdir(image_folder)):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(image_folder, filename)
            transparent = remove_background(image_path)
            cv2.imwrite("transparent_" + str(i) + ".png", transparent, [cv2.IMWRITE_PNG_COMPRESSION, 9])

process_images("/content/images")


In [6]:
import cv2
import numpy as np

def remove_background(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pixels = img.reshape(-1, 3).astype(np.float32)

    k = 2
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    background_pixels = pixels[labels.flatten() == 0]
    mean_background = np.mean(background_pixels, axis=0).astype(np.uint8)

    mask = np.zeros(img.shape[:2], np.uint8)
    mask[np.all(np.abs(img - mean_background) < 15, axis=-1)] = 255
    mask = cv2.medianBlur(mask, 7)

    foreground = cv2.bitwise_and(img, img, mask=mask)
    background = np.full(img.shape, mean_background, dtype=np.uint8)
    background = cv2.bitwise_and(background, background, mask=cv2.bitwise_not(mask))

    transparent = cv2.addWeighted(background, 0.5, foreground, 0.5, 0)
    transparent = cv2.cvtColor(transparent, cv2.COLOR_RGB2RGBA)
    _, alpha = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)
    b, g, r, a = cv2.split(transparent)
    rgba = [b, g, r, alpha]
    transparent = cv2.merge(rgba, 4)

    return transparent

def process_images(image_folder):
    for i, filename in enumerate(os.listdir(image_folder)):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(image_folder, filename)
            transparent = remove_background(image_path)
            cv2.imwrite(os.path.join(image_folder, "transparent_" + str(i) + ".png"), transparent, [cv2.IMWRITE_PNG_COMPRESSION, 9])

process_images("/content/images/")


In [9]:
import cv2
import numpy as np
import os

def remove_background(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pixels = np.float32(image.reshape(-1, 3))

    k = 2
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
    _, labels, (centers) = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    center = np.uint8(centers)
    labels = labels.flatten()
    background_label = 0
    background_pixels = pixels[labels == background_label]
    mean = np.mean(background_pixels, axis=0)
    background_color = np.uint8(mean)

    mask = labels != background_label
    image = image.reshape(-1, 3)
    image = np.uint8(image)
    image[mask] = image[mask]
    image = image.reshape(image.shape[0], image.shape[1], -1)

    transparent = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)
    transparent[:, :, :3] = image
    mask = mask.reshape((mask.shape[0], 1))
    transparent[:, :, 3] = mask.astype(int) * 255


    return transparent

image_folder = "/content/images"
output_folder = "/content/output"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i, filename in enumerate(os.listdir(image_folder)):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        image_path = os.path.join(image_folder, filename)
        transparent = remove_background(image_path)
        output_path = os.path.join(output_folder, "transparent_" + str(i) + ".png")
        cv2.imwrite(output_path, transparent)


In [10]:
import cv2
import numpy as np
import os

def remove_background(img_path):
    # Load the image
    img = cv2.imread(img_path)

    # Define a rectangle that covers the object in focus
    rect = (50, 50, img.shape[1]-100, img.shape[0]-100)

    # Define the mask for the GrabCut algorithm
    mask = np.zeros(img.shape[:2], np.uint8)
    bg_model = np.zeros((1, 65), np.float64)
    fg_model = np.zeros((1, 65), np.float64)

    # Run the GrabCut algorithm to segment the image
    cv2.grabCut(img, mask, rect, bg_model, fg_model, 5, cv2.GC_INIT_WITH_RECT)

    # Create a binary mask with the segmentation results
    mask = np.where((mask==2)|(mask==0), 0, 1).astype('uint8')
    img = img * mask[:,:,np.newaxis]

    # Save the output as a transparent PNG
    alpha = np.ones(img.shape[:2], dtype=np.uint8) * 255
    transparent = np.dstack((img, alpha))
    cv2.imwrite(f"{os.path.splitext(img_path)[0]}_transparent.png", transparent)

# Example usage:
remove_background("/content/images/dgsdgsd.jpg")
remove_background("/content/images/images.jpeg")


In [13]:
import cv2
import numpy as np
import os

# Read images from the directory
images_path = "/content/images"
images = [f for f in os.listdir(images_path) if f.endswith('.jpg') or f.endswith('.png')]

for image_name in images:
    image_path = os.path.join(images_path, image_name)
    image = cv2.imread(image_path)
    h, w, c = image.shape

    # Convert image to LAB color space and separate channels
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply k-means clustering to the L channel
    k = 5
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    pixels = l.reshape(-1, 1)
    _, labels, (centers) = cv2.kmeans(pixels.astype(np.float32), k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Create a binary mask based on the k-means result
    mask = np.zeros(l.shape[:2], dtype=np.uint8)
    for i in range(k):
        if centers[i] > 128:
            labels = labels.reshape(mask.shape)
            mask[labels == i] = 255
    
    # Remove the out-of-focus parts using the binary mask
    result = cv2.bitwise_and(image, image, mask=mask)

    # Create a transparent background
    transparent = np.zeros((h, w, 4), dtype=np.uint8)
    transparent[:, :, :3] = result
    transparent[:, :, 3] = mask.astype(int) * 255

    # Save the image as a transparent PNG
    save_path = os.path.join(images_path, image_name.split(".")[0] + "_result.png")
    cv2.imwrite(save_path, transparent)


In [14]:
import cv2
import numpy as np
import os

path = "/content/images/"

for filename in os.listdir(path):
    image = cv2.imread(os.path.join(path, filename))
    if image is None:
    print("Error: Image not loaded.")
else:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    ret, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    mask = np.dstack([mask, mask, mask]) / 255
    h, w, c = image.shape
    mask, bg = np.zeros((h, w), np.uint8), np.zeros((h, w), np.uint8)
    rect = (1, 1, w-2, h-2)
    cv2.grabCut(image, mask, rect, bg, fg, 5, cv2.GC_INIT_WITH_RECT)
    mask = np.where((mask == 2)|(mask == 0), 0, 1).astype('uint8')
    image = image * mask[:, :, np.newaxis]
    bg = np.ones_like(image, np.uint8) * 255
    final = cv2.addWeighted(image, 0.7, bg, 0.3, 0)
    cv2.imwrite(os.path.join(path, "result_" + filename), final)


IndentationError: ignored

In [15]:
import os
import cv2
import numpy as np

def remove_background(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Perform Otsu thresholding to create a binary mask
    ret, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Define the mask for the GrabCut algorithm
    mask = np.where((mask == 0), 0, 1).astype('uint8')
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    # Perform GrabCut to remove the out-of-focus parts
    rect = (0, 0, image.shape[1], image.shape[0])
    cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_MASK)

    # Create a transparent image
    transparent = np.zeros_like(image, dtype=np.uint8)
    transparent[:, :, :3] = image
    transparent[:, :, 3] = mask.astype(int) * 255

    return transparent

# Define the path to the images
image_folder = '/content/images/'

# Loop through all the images in the folder
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg'):
        # Load the image
        image = cv2.imread(os.path.join(image_folder, filename))

        # Remove the out-of-focus parts and create a transparent image
        transparent = remove_background(image)

        # Save the transparent image as a PNG file
        cv2.imwrite(os.path.join(image_folder, 'transparent_' + filename.replace('.jpg', '.png')), transparent)


IndexError: ignored

In [16]:
import cv2
import numpy as np
import os

path = "/content/images/"

for filename in os.listdir(path):
    image = cv2.imread(os.path.join(path,filename))
    
    if image is None:
        print("Error: Image not loaded.")
    else:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Perform Otsu thresholding
        thresh, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # Define the mask for the GrabCut algorithm
        mask = np.where((mask==0), 0, 1).astype('uint8')
        bgModel = np.zeros((1,65),np.float64)
        fgModel = np.zeros((1,65),np.float64)
        
        # Perform GrabCut
        x,y,w,h = cv2.boundingRect(mask)
        rect = (x,y,w,h)
        bgdModel = np.zeros((1,65),np.float64)
        fgdModel = np.zeros((1,65),np.float64)
        cv2.grabCut(image,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
        mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        
        # Create a transparent image
        transparent = np.dstack((image, np.ones_like(mask)*255))
        transparent[:,:,3] = mask*255
        
        # Save the result
        cv2.imwrite(os.path.join(path, "result_" + filename), transparent)


Error: Image not loaded.


error: ignored

In [17]:
import cv2
import numpy as np
import os

# Define the path to the images
path = "/content/images/"

for filename in os.listdir(path):
    # Load the image
    image = cv2.imread(path + filename)

    # Check if the image was loaded successfully
    if image is None:
        print("Error: Image not loaded.")
        continue
    else:
        # Convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Perform Otsu thresholding
        ret,threshold = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        # Define the mask for the GrabCut algorithm
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        bgdModel = np.zeros((1,65),np.float64)
        fgdModel = np.zeros((1,65),np.float64)
        rect = (0,0,image.shape[1]-1,image.shape[0]-1)

        # Perform GrabCut segmentation
        cv2.grabCut(image,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_MASK)

        # Create the binary mask
        mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')

        # Apply the binary mask to the image
        transparent = cv2.cvtColor(image, cv2.COLOR_BGR2RGBA)
        transparent[:, :, 3] = mask.astype(int) * 255

        # Save the output
        cv2.imwrite(path + "output_" + filename, transparent)


Error: Image not loaded.


error: ignored

In [18]:
import cv2
import numpy as np
import os

def get_transparent_mask(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Define the mask for the GrabCut algorithm
    mask = np.zeros(image.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    rect = (0, 0, image.shape[1] - 1, image.shape[0] - 1)

    # Perform GrabCut
    cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_MASK)

    # Remove the out-of-focus parts using the binary mask
    mask = np.where((mask == cv2.GC_BGD) | (mask == cv2.GC_PR_BGD), 0, 1).astype('uint8')
    return mask

def remove_background(image):
    mask = get_transparent_mask(image)
    transparent = np.zeros_like(image)
    transparent[:, :, :3] = image[:, :, :3]
    transparent[:, :, 3] = mask.astype(int) * 255
    return transparent

path = '/content/images'
images = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

for image_name in images:
    image_path = os.path.join(path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Image {image_path} not loaded.")
        continue
    transparent = remove_background(image)
    cv2.imwrite(f"transparent_{image_name}", transparent)


error: ignored

In [19]:
import cv2
import numpy as np
import os

# Define the path to the images
path = "/content/images/"

for file in os.listdir(path):
  # Load the image
  image = cv2.imread(os.path.join(path, file))

  # Convert the image to grayscale
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Perform Otsu thresholding
  _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

  # Define the mask for the GrabCut algorithm
  mask = np.zeros(image.shape[:2], np.uint8)
  bgdModel = np.zeros((1, 65), np.float64)
  fgdModel = np.zeros((1, 65), np.float64)

  # Define the rectangle to be used as the foreground
  x, y, w, h = cv2.boundingRect(thresh)
  rect = (x, y, w, h)

  # Perform the GrabCut algorithm
  cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

  # Remove the out-of-focus parts using the binary mask
  mask = np.where((mask == 2) | (mask == 0), 0, 1).astype("uint8")
  image = image * mask[:, :, np.newaxis]

  # Convert the image to RGBA format and make the background transparent
  transparent = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
  transparent[:, :, 3] = mask.astype(int) * 255

  # Save the transparent image
  cv2.imwrite(os.path.join(path, "transparent_" + file), transparent)


error: ignored